<a href="https://colab.research.google.com/github/alessandro-rubin/databricks_training/blob/main/ms_aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9087ff39b3b170b26cf362da9369ae547e67b19b3ed6b8b2b6447072e7654f51
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from datetime import datetime, timedelta
import random
from pyspark.sql.functions import window, col
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructType, StructField, TimestampType

from pyspark.sql import Window

In [16]:


# Create a Spark session
spark = SparkSession.builder.appName("ColabSparkSession").getOrCreate()

# Define the schema for the DataFrame
schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("VIN", StringType(), True),
    StructField("odometer", DoubleType(), True),
    StructField("rail_pressure", DoubleType(), True)
])

# Generate random data for each VIN
num_vins = 10
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 8)
interval_in_seconda=30
interval = timedelta(seconds=interval_in_seconda)

data = []
for vin_index in range(num_vins):
    vin = f"VIN_{vin_index}"
    start_odometer = random.uniform(1000, 2000)
    end_odometer = random.uniform(start_odometer, start_odometer + 1000)
    avg_pressure = random.uniform(5, 7)

    current_date = start_date
    while current_date <= end_date:
        data.append((current_date, vin, start_odometer, random.gauss(avg_pressure, 2)))
        current_date += interval
        start_odometer += (end_odometer - start_odometer) / ((end_date - start_date).total_seconds() / 60)

# Create the DataFrame
df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()



+-------------------+-----+------------------+------------------+
|          timestamp|  VIN|          odometer|     rail_pressure|
+-------------------+-----+------------------+------------------+
|2023-01-01 00:00:00|VIN_0|1899.2357124788582| 3.603427405618819|
|2023-01-01 00:00:30|VIN_0|1899.3260237195163| 6.508120407023222|
|2023-01-01 00:01:00|VIN_0| 1899.416326000726| 5.841965642892393|
|2023-01-01 00:01:30|VIN_0| 1899.506619323376| 8.414535954296237|
|2023-01-01 00:02:00|VIN_0|1899.5969036883553| 7.442419208452327|
|2023-01-01 00:02:30|VIN_0|1899.6871790965522| 6.019580028720641|
|2023-01-01 00:03:00|VIN_0|1899.7774455488554|3.8091239517834214|
|2023-01-01 00:03:30|VIN_0|1899.8677030461536| 7.394373718949166|
|2023-01-01 00:04:00|VIN_0| 1899.957951589335| 5.377180543595734|
|2023-01-01 00:04:30|VIN_0|1900.0481911792876| 3.433761064232664|
|2023-01-01 00:05:00|VIN_0|   1900.1384218169|4.1705455324491325|
|2023-01-01 00:05:30|VIN_0|1900.2286435030605|4.5664483449680215|
|2023-01-0

In [21]:
df=df.withColumn('MS_1',F.when(col('rail_pressure')>7,1).otherwise(0))
df=df.withColumn("hourly_timestamp", F.date_trunc("hour", df.timestamp))
df.show()

+-------------------+-----+------------------+------------------+----+-------------------+
|          timestamp|  VIN|          odometer|     rail_pressure|MS_1|   hourly_timestamp|
+-------------------+-----+------------------+------------------+----+-------------------+
|2023-01-01 00:00:00|VIN_0|1899.2357124788582| 3.603427405618819|   0|2023-01-01 00:00:00|
|2023-01-01 00:00:30|VIN_0|1899.3260237195163| 6.508120407023222|   0|2023-01-01 00:00:00|
|2023-01-01 00:01:00|VIN_0| 1899.416326000726| 5.841965642892393|   0|2023-01-01 00:00:00|
|2023-01-01 00:01:30|VIN_0| 1899.506619323376| 8.414535954296237|   1|2023-01-01 00:00:00|
|2023-01-01 00:02:00|VIN_0|1899.5969036883553| 7.442419208452327|   1|2023-01-01 00:00:00|
|2023-01-01 00:02:30|VIN_0|1899.6871790965522| 6.019580028720641|   0|2023-01-01 00:00:00|
|2023-01-01 00:03:00|VIN_0|1899.7774455488554|3.8091239517834214|   0|2023-01-01 00:00:00|
|2023-01-01 00:03:30|VIN_0|1899.8677030461536| 7.394373718949166|   1|2023-01-01 00:00:00|

In [20]:
# Run the aggregation code provided in the previous message
window_spec = window(col("timestamp"), "1 hour")
result = df.groupBy(window_spec, "vin").agg(
    F.sum(col("MS_1")).alias("binary_column_1_sum"),
    F.count("*").alias("total_rows_in_window")
)

result=result.withColumn("Win_start",col("window.start"))
result.orderBy(col("window.start").asc(),col('vin')).show()

+--------------------+-----+-------------------+--------------------+-------------------+
|              window|  vin|binary_column_1_sum|total_rows_in_window|          Win_start|
+--------------------+-----+-------------------+--------------------+-------------------+
|{2023-01-01 00:00...|VIN_0|                 25|                 120|2023-01-01 00:00:00|
|{2023-01-01 00:00...|VIN_1|                 25|                 120|2023-01-01 00:00:00|
|{2023-01-01 00:00...|VIN_2|                 43|                 120|2023-01-01 00:00:00|
|{2023-01-01 00:00...|VIN_3|                 34|                 120|2023-01-01 00:00:00|
|{2023-01-01 00:00...|VIN_4|                 38|                 120|2023-01-01 00:00:00|
|{2023-01-01 00:00...|VIN_5|                 28|                 120|2023-01-01 00:00:00|
|{2023-01-01 00:00...|VIN_6|                 21|                 120|2023-01-01 00:00:00|
|{2023-01-01 00:00...|VIN_7|                 47|                 120|2023-01-01 00:00:00|
|{2023-01-

In [22]:
df.groupBy('vin','hourly_timestamp').agg(F.sum('MS_1'),F.count('*')).orderBy(['hourly_timestamp','vin']).show()

+-----+-------------------+---------+--------+
|  vin|   hourly_timestamp|sum(MS_1)|count(1)|
+-----+-------------------+---------+--------+
|VIN_0|2023-01-01 00:00:00|       25|     120|
|VIN_1|2023-01-01 00:00:00|       25|     120|
|VIN_2|2023-01-01 00:00:00|       43|     120|
|VIN_3|2023-01-01 00:00:00|       34|     120|
|VIN_4|2023-01-01 00:00:00|       38|     120|
|VIN_5|2023-01-01 00:00:00|       28|     120|
|VIN_6|2023-01-01 00:00:00|       21|     120|
|VIN_7|2023-01-01 00:00:00|       47|     120|
|VIN_8|2023-01-01 00:00:00|       24|     120|
|VIN_9|2023-01-01 00:00:00|       50|     120|
|VIN_0|2023-01-01 01:00:00|       24|     120|
|VIN_1|2023-01-01 01:00:00|       18|     120|
|VIN_2|2023-01-01 01:00:00|       39|     120|
|VIN_3|2023-01-01 01:00:00|       38|     120|
|VIN_4|2023-01-01 01:00:00|       39|     120|
|VIN_5|2023-01-01 01:00:00|       33|     120|
|VIN_6|2023-01-01 01:00:00|       23|     120|
|VIN_7|2023-01-01 01:00:00|       35|     120|
|VIN_8|2023-0